In [32]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d, KDTree
from tqdm import tqdm
from random import random
from math import sqrt 

# Load image
# image_path = "/Users/nshelton/Pictures/daruma.jpg"
image_path = "/Users/nshelton/Pictures/majora.jpg"
# image_path = "/Users/nshelton/Desktop/rozo2.png"
# image_path = "/Users/nshelton/Desktop/calligraphy.png"
# image_path = "/Users/nshelton/Pictures/Gray093.jpg"

# image_path = "/Users/nshelton/Pictures/horsemen.png"
image = cv2.imread(image_path)
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# image_value = cv2.resize(image_gray, (-1, -1), fx = 0.5, fy = 0.5)
image_value = image_gray
image_value = (255 - image_value).astype(np.float32)
image_value -= image_value.min()

image_value /= image_value.max()

filtered = image_value * 255
# filtered = cv2.blur(filtered, (7,7)) 
# filtered = np.abs(cv2.Laplacian(filtered * 50, cv2.CV_32F))

filtered -= filtered.min()
filtered /= filtered.max()

filtered = (255 * filtered).astype(np.uint8) 
# filtered= cv2.adaptiveThreshold(filtered, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, -20)



filtered = np.pad(filtered, 10, mode='constant', constant_values=0)
# filtered = cv2.medianBlur(filtered, 3)
cv2.imwrite("filtered.png", filtered)



print(filtered.shape, filtered.dtype, filtered.min(), filtered.max(), filtered.mean())

(818, 584) uint8 0 255 126.14138853535184


In [38]:
from pathUtils import *

remaining_ink = filtered.copy()

all_strokes_img = np.ones(remaining_ink.shape) * 255
all_paths = []

MAX_PATHS = 300000

for i in range(MAX_PATHS):
    if remaining_ink.sum() == 0:
        break

    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(remaining_ink)

    # print(max_loc)
    line_thickness = 3
    stroke_color = 64
    window = 1

    def lookupxy(raster, xy):
        if int(xy[1]) > 0 and int(xy[1]) < raster.shape[0] and  int(xy[0]) > 0 and int(xy[0]) < raster.shape[1]:
            return raster[int(xy[1]), int(xy[0])]
        return np.nan
    
    def placeStroke(input_raster, start):
        img = input_raster.copy()
        path = []
        pos = np.array(start)

        max_val = lookupxy(input_raster, pos)
        start_val = max_val
        dir = np.array([1, 0])
        while True:
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(
                img[
                    round(pos[1] - window + dir[1]): round(pos[1] + window + 1 + dir[1]),
                    round(pos[0] - window + dir[0]): round(pos[0] + window + 1 + dir[0]),
                ]
            )
            if max_val < start_val/2:
                return path
            dir = dir * 0.9 + (np.array(max_loc) - window) * 0.1
            pos += np.array(max_loc) - window
            path.append(pos.copy())
            cv2.circle(img, pos, window, 0, -1)

    
    color = stroke_color
    def rasterStroke(dims, path):

        img = np.zeros(dims)

        if (len(path)) == 1:
            cv2.circle(
                img, (round(path[0][0]), round(path[0][1])), line_thickness//2, color, -1
            )

        for i in range(len(path) - 1):
            a = (round(path[i][0]), round(path[i][1]))
            b = (round(path[i + 1][0]), round(path[i + 1][1]))
            cv2.line(img, a, b, color, line_thickness, cv2.LINE_AA)

        return img
    def smooth_stroke(path):
        iterations = 10
        for j in range(iterations):
            for i in range(1, len(path)-1):
                a = path[i-1]
                b = path[i+1]
                c = (a+b)/2
                path[i] = path[i] * 0.5 + c * 0.5

        return path

    
    new_stroke = placeStroke(remaining_ink, max_loc)
    new_stroke = smooth_stroke(new_stroke)
    all_paths = add_path_with_merge(all_paths, new_stroke, threshold = 2)
    
    stroke_raster = rasterStroke(filtered.shape, new_stroke)

    remaining_ink = remaining_ink - stroke_raster
    remaining_ink = np.clip(remaining_ink, 0, 255)
    all_strokes_img = np.clip(all_strokes_img.astype(np.int32) - stroke_raster.astype(np.int32) * 100, 0,255).astype(np.uint8)
    if len(all_paths) % 100 == 0:
        cv2.imwrite("current_stroke.png",  (all_strokes_img).astype(np.uint8))
        cv2.imwrite("remaining_ink.png", remaining_ink)

    print(len(all_paths), remaining_ink.sum(), remaining_ink.max(), remaining_ink.min(), end = "\r")




KeyboardInterrupt: 

In [39]:
from tqdm import tqdm


path_list = [[p for p in path] for path in all_paths if len(path) > 1]

print(len(all_paths))
print(len(path_list))

Merge = True

if Merge:
    merge_threshold = 1

    for i in tqdm(range(len(path_list))):
        len_before = len(path_list)
        path_list = merge_one_path_kd(path_list, merge_threshold)
        if len_before == len(path_list):

            break

# def getPathLength(path):
    # length = 0g
    # for i in range(len(path) - 1):
        # length += np.linalg.norm(path[i] - path[i+1])
    # return length

# path_list = close_loops(path_list, threshold=0.01)
path_list  = [np.array(path) for path in path_list]

# path_list = [path for path in path_list if getPathLength(path) > 2]
# path_list = smooth_all_2(path_list, 0.01)

path_list = simplify_all(path_list, 0.001)
# path_list = smooth_all_2(path_list, 0.01)

# path_list = simplify_all(path_list, 0.1)

RENDER_SCALE = 0.5
render_img = plotPaths(path_list, RENDER_SCALE)
cv2.imwrite("render.png", render_img)

1259
1259


  2%|▏         | 25/1259 [00:00<00:23, 53.08it/s]


True

In [27]:
 
import json
import uuid

projects_folder = "/Users/nshelton/Hephaestus/projects"
# projects_folder = "."

def writePlot(paths):
    dictionary = {
        "created_time": "4/7/2024 4:42:27 PM",
        "modified_time": "4/7/2024 6:27:36 PM",
        "camera_position": [100, 100],
        "zoom": 100.702,
        "aspect": 0.563,
        "dom_element": {},
        "plot_models": [],
    }

    dictionary["plot_models"].append(
        {
            "position": {"x": 0, "y": 0},
            "paths": [path.tolist() for path in path_list],
            "scale": 0.1,
            "id": str(uuid.uuid4()),
            "state": "none",
        }
    )

    with open(projects_folder + "/vetebrae_squirl.json", "w") as outfile:
        
        json.dump(dictionary, outfile)

writePlot(path_list)

